In [119]:
import sys
import os

# Add the parent directory (project root) to sys.path
sys.path.append(os.path.abspath('..'))

from sandbox_steps.main import DesktopSteps
from e2b_desktop import Sandbox, CommandExitException
E2B_API_KEY = os.environ.get("E2B_API_KEY")

In [28]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

https://6080-iaqgg27k5t5orpequpb6q-ae06b565.e2b.app/vnc.html?autoconnect=true&resize=scale


In [121]:
running_sandboxes = Sandbox.list()
running_sandboxes

[]

In [8]:
Sandbox.kill("ig67ifl7itvszhdnlxm3h-6cc9e88b")

True

In [ ]:
Sandbox.left_click(100, 100) 
Sandbox.right_click(200, 200)

In [127]:
def install_csgo(desktop: Sandbox):
    desktop.open("https://www.cybersports.lt/setup/")
    desktop.commands.run("sudo dpkg --add-architecture i386")
    desktop.commands.run("sudo apt update")
    desktop.wait(100)
    desktop.commands.run("sudo apt install -y wine32")
    desktop.wait(2000)
    try:
        desktop.commands.run("wine .") # this throws an error for fun
    except CommandExitException:
        pass
    desktop.open("https://drive.google.com/u/0/uc?id=1TIsvGACSrQOr1tgPaVpJebH375LjLIV6&export=download")
    desktop.wait(ms=4000)
    desktop.left_click(460, 305) #click the Download button
    desktop.wait(ms=5000) 
    desktop.commands.run("unzip Downloads/Windows7DefaultFonts.zip -d Downloads")
    desktop.commands.run("mv Downloads/Windows7DefaultFonts/* ~/.wine/drive_c/windows/Fonts/")
    desktop.left_click(880, 1040) # open the terminal (I know, clean)
    desktop.wait(100)
    desktop.left_click(1800, 800) # open the terminal (I know, clean)
    desktop.write("wine Downloads/Counter-Strike-1.6-original.exe", chunk_size=50, delay_in_ms=25) 
    desktop.wait(2000)
    desktop.press("enter") # start the exe with wine
    desktop.wait(6000)
    desktop.press("enter") # start the installation
    desktop.wait(2000)
    desktop.press("enter") # one menu
    desktop.wait(2000)
    desktop.press("enter") # second menu
    desktop.wait(2000)
    desktop.press("enter") # final install button
    desktop.wait(55000) # installation process
    desktop.press("enter") # LAUNCH


In [ ]:
desktop.left_click(460, 305) # click on the button to download fonts

In [128]:
install_csgo(desktop=desktop)

In [129]:
desktop.kill()